コストラベル作成器

16bit mapのコストラベル

In [139]:
import time
import glob
import pickle
import numpy as np
import pandas as pd
from pandarallel import pandarallel
# from tqdm import tqdm

from ifloat.usecase.area_evaluator import SplittingGridEvaluator, WholeGridEvaluator, IAreaEvaluator
from ifloat.util.dependency import Dependency
from ifloat.infra.visualizer import visualize_floor, visualize_grid_and_sg

In [140]:
injector = Dependency()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [141]:
with open("data_set/layout_test.pkl", "rb") as f:
    _, area, routes = pickle.load(f)

routes = routes[:-1]

訓練データのロード

In [142]:
with open("data_set/training/facilities_test_0001.pkl", "rb") as f:
    train_f_dataset = pickle.load(f)

テストデータのロード

In [143]:
with open("data_set/test/facilities_test_0001.pkl", "rb") as t:
    test_f_dataset = pickle.load(t)

検証データのロード

In [144]:
with open("data_set/valid/facilities_test_0001.pkl", "rb") as f:
    V_facility_dataset = pickle.load(f)

訓練データをデータフレーム化

In [145]:
train_df = pd.DataFrame()
train_df["facilities"] = train_f_dataset



print(f"dataset number: {len(train_df)}")

dataset number: 10000


テストデータのデータフレーム化

In [146]:
test_df = pd.DataFrame()
test_df["facilities"] = test_f_dataset

# df = df[:2000]

print(f"dataset number: {len(test_df)}")

dataset number: 1000


コスト計算器

In [147]:
def evaluate_entirely(area, facilities, routes):
    evaluator: WholeGridEvaluator = injector.resolve(WholeGridEvaluator)

    t0 = time.time()
    result = evaluator.evaluate_objects(area, facilities, routes)
    t1 = time.time()

    return result.total_travel_time, t1 - t0

# for i in [11]:
#     facilities = df.iloc[i]["facilities"]
#     print(i)
#     print(evaluate_entirely(area, facilities, routes))

# for i in range(0, 10):
#     facilities = df.iloc[i]["facilities"]
#     print(i)
#     print(area,facilities, routes)
#     print(evaluate_entirely(area, facilities, routes))

各マップのデータを格納

In [148]:
from mlmodel.mk_model import make_deep_data,make_dataset,make_deep_data_s_g

In [149]:
map_only_data,starts_data,goals_data=make_dataset(area,train_df,routes)

レイヤー統合して入力データに変換

In [150]:
# train_in_deep_data = make_deep_data(map_only_data,starts_data,goals_data)

In [151]:
def create_map_with_agents_sg2chanel(map_data, starts,goals):#スタートゴールを全体の1,0にして格納
    
    
    # マップのサイズを取得
    map_height, map_width = 16,16
    
    # レイヤーを作成：マップ + エージェントのスタートとゴール
    num_agents = len(starts)
    map_with_agents = np.zeros((map_height, map_width,  num_agents+1))
    
    # ベースのマップ情報を最初のレイヤーに配置
    map_with_agents[:, :, 0] = map_data
    
    # 各エージェントのスタートとゴールを配置
    for i in range(len(starts)):
        start_x = starts[i][0]
        start_y = starts[i][1]
        # スタート位置を (i*2 + 1) レイヤーに設定
        map_with_agents[start_x,start_y, i+1] = 1
        
        goal_x = goals[i][0]
        goal_y = goals[i][1]
        # ゴール位置を (i*2 + 2) レイヤーに設定
        map_with_agents[goal_x,goal_y, i+1] = 1
    
    return map_with_agents


def make_deep_data_s_g(maps,starts,goals):#マップ、スタート、ゴールをレイヤー統合してリストを出力
    in_deep_data =[]
    
    
    for i in range(len(maps)):
        
        map_with =create_map_with_agents_sg2chanel(maps[i],starts[i],goals[i])

        in_deep_data.append(map_with)
        
        
    return in_deep_data

In [152]:

train_in_deep_data = make_deep_data_s_g(map_only_data,starts_data,goals_data)

traina=np.array(train_in_deep_data)

print(traina.shape)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(10000, 16, 16, 4)


In [153]:
# 到達時間と計算時間
train_df[["cost_entirely","time_entirely"]] = train_df.apply(
    lambda r: evaluate_entirely(area, r["facilities"], routes),
    axis=1,
    result_type="expand"
)


無限の外れ値を最大値に置き換え

In [154]:

max_cost= max(train_df["cost_entirely"])

print(max_cost)

if max_cost==np.inf :
    filtered_values = [x for x in train_df["cost_entirely"] if x != np.inf]
    
    max_cost=max(filtered_values)


print(max_cost)

#無限のコストにならないように上限を設ける
train_df["cost_entirely"]= np.clip(train_df["cost_entirely"],0,max_cost)


# train_label=[]
# train_label=train_df["cost_entirely"]


# print(train_df)

inf
101.0


マップデータを保存

In [155]:
with open("data_set/training/2chan_map_data.pkl", "wb") as f:
    pickle.dump(train_in_deep_data, f)

コストラベル

In [156]:
# cost_label=[]
# for i in range(len(train_df)):
#     cost_label.append(train_df.iloc[i]["cost_entirely"])




In [157]:
cost_label=[]
for i in range(len(train_df)):
    c_label =[]
    
    c_label.append(train_df.iloc[i]["cost_entirely"])
    
    cost_label.append(c_label)
    
    
# print(cost_label)

In [158]:
with open("data_set/training_label/2chan_solo_cost_labels.pkl", "wb") as f:
    pickle.dump(cost_label, f)

テストデータの作成

In [159]:
test_map_only_data,test_starts_data,test_goals_data=make_dataset(area,test_df,routes)

In [160]:
# test_in_deep_data = make_deep_data(test_map_only_data,test_starts_data,test_goals_data)

In [161]:
test_in_deep_data = make_deep_data_s_g(test_map_only_data,test_starts_data,test_goals_data)

In [162]:
# 到達時間と計算時間
test_df[["cost_entirely","time_entirely"]] = test_df.apply(
    lambda r: evaluate_entirely(area, r["facilities"], routes),
    axis=1,
    result_type="expand"
)


In [163]:
max_cost= max(test_df["cost_entirely"])

print(max_cost)

if max_cost==np.inf :
    filtered_values = [x for x in test_df["cost_entirely"] if x != np.inf]
    
    max_cost=max(filtered_values)


print(max_cost)
#無限のコストにならないように上限を設ける
test_df["cost_entirely"]= np.clip(test_df["cost_entirely"],0,max_cost)

test_label=[]
test_label=test_df["cost_entirely"]
# print(test_df)

inf
80.0


In [164]:
sum_time =test_df.sum(axis=0)
print(sum_time)

facilities       [Facility(name='f1', pose=Pose3D(position=Poin...
cost_entirely                                              51150.0
time_entirely                                             3.102989
dtype: object


テストデータのコストのラベル付け

In [165]:
test_cost_label=[]
for i in range(len(test_df)):
    m_label =[]
    
    m_label.append(test_df.iloc[i]["cost_entirely"])
    
    test_cost_label.append(m_label)
    
    
# print(test_cost_label)

マップデータの保存

In [166]:
# with open("data_set/test/map_data.pkl", "wb") as f:
#     pickle.dump(test_in_deep_data, f)
    
with open("data_set/test/2chan_map_data.pkl", "wb") as f:
    pickle.dump(test_in_deep_data, f)

ラベルの保存

In [167]:
# with open("data_set/test_label/solo_cost_labels.pkl", "wb") as f:
#     pickle.dump(test_cost_label, f)
#n chanel
    
with open("data_set/test_label/2chan_solo_cost_labels.pkl", "wb") as f:
    pickle.dump(test_cost_label, f)
    #2chanel

検証データのdf化

In [168]:
valid_df = pd.DataFrame()
valid_df["facilities"] = V_facility_dataset

検証データの作成

In [169]:
valid_map_only_data,valid_starts_data,valid_goals_data=make_dataset(area,valid_df,routes)

# valid_in_deep_data = make_deep_data(valid_map_only_data,valid_starts_data,valid_goals_data)

valid_in_deep_data = make_deep_data_s_g(valid_map_only_data,valid_starts_data,valid_goals_data)

In [170]:
# 到達時間と計算時間
valid_df[["cost_entirely","time_entirely"]] = valid_df.apply(
    lambda r: evaluate_entirely(area, r["facilities"], routes),
    axis=1,
    result_type="expand"
)

In [171]:
V_max_cost= max(valid_df["cost_entirely"])

print(V_max_cost)

if V_max_cost==np.inf :
    filtered_values = [x for x in valid_df["cost_entirely"] if x != np.inf]
    
    V_max_cost=max(filtered_values)


print(V_max_cost)
#無限のコストにならないように上限を設ける
valid_df["cost_entirely"]= np.clip(valid_df["cost_entirely"],0,V_max_cost)

# valid_label=[]
# valid_label=valid_df["cost_entirely"]
# print(test_df)

inf
82.0


In [172]:
valid_cost_label=[]
for i in range(len(valid_df)):
    v_label =[]
    
    v_label.append(valid_df.iloc[i]["cost_entirely"])
    
    valid_cost_label.append(v_label)
    
    
print(valid_cost_label)

[[27.0], [37.0], [33.0], [38.0], [65.0], [45.0], [46.0], [52.0], [54.0], [52.0], [38.0], [77.0], [44.0], [44.0], [41.0], [55.0], [57.0], [36.0], [55.0], [50.0], [52.0], [54.0], [53.0], [67.0], [44.0], [44.0], [48.0], [39.0], [38.0], [45.0], [56.0], [52.0], [54.0], [53.0], [38.0], [63.0], [70.0], [42.0], [69.0], [57.0], [63.0], [52.0], [59.0], [54.0], [39.0], [57.0], [48.0], [38.0], [60.0], [50.0], [51.0], [53.0], [41.0], [47.0], [48.0], [53.0], [55.0], [62.0], [58.0], [46.0], [55.0], [55.0], [51.0], [29.0], [37.0], [47.0], [39.0], [71.0], [29.0], [53.0], [43.0], [43.0], [70.0], [52.0], [73.0], [39.0], [50.0], [56.0], [41.0], [65.0], [79.0], [41.0], [45.0], [42.0], [53.0], [48.0], [30.0], [64.0], [57.0], [67.0], [52.0], [49.0], [45.0], [50.0], [55.0], [56.0], [50.0], [58.0], [53.0], [50.0], [52.0], [62.0], [65.0], [52.0], [39.0], [47.0], [37.0], [55.0], [52.0], [54.0], [53.0], [43.0], [64.0], [59.0], [47.0], [41.0], [64.0], [52.0], [52.0], [44.0], [63.0], [48.0], [42.0], [50.0], [54.0],

検証データの保存

In [173]:
# with open("data_set/valid/map_data.pkl", "wb") as f:
#     pickle.dump(valid_in_deep_data, f)

with open("data_set/valid/2chan_map_data.pkl", "wb") as f:
    pickle.dump(valid_in_deep_data, f)

検証ラベルの保存

In [174]:
# with open("data_set/valid_label/solo_cost_labels.pkl", "wb") as f:
#     pickle.dump(valid_cost_label, f)
#n chanel

with open("data_set/valid_label/2chan_solo_cost_labels.pkl", "wb") as f:
    pickle.dump(valid_cost_label, f)